In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Sashank Ram\\Documents\\End-to-end-ML-Final-with-MLFlo\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Sashank Ram\\Documents\\End-to-end-ML-Final-with-MLFlo'

In [27]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/saz12345/End-to-end-ML-Final-with-MLFlo.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="saz12345"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1dec117f94c57343f27a2ed36ecf54154970d635"

In [34]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [35]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [36]:
from src.mlProject.constants._init_ import CONFIG_FILE_PATH
from src.mlProject.constants._init_ import PARAMS_FILE_PATH
from src.mlProject.constants._init_ import SCHEMA_FILE_PATH

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params =read_yaml(params_filepath)
            self.schema =read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
          config = self.config.model_evaluation
          params= self.params.ElasticNet
          schema= self.schema.TARGET_COLUMN

          create_directories([config.root_dir])

          model_evaluation_config= ModelEvaluationConfig(
              root_dir= Path(config.root_dir),
              test_data_path= Path(config.test_data_path),
              model_path= Path(config.model_path),
              all_params=params,
              metric_file_name= Path(config.metric_file_name),
              target_column= schema.name,
              mlflow_uri="https://dagshub.com/saz12345/End-to-end-ML-Final-with-MLFlo.mlflow"
    )

          return model_evaluation_config

In [37]:
import pandas as pd
import os
from src.mlProject.utils._init_ import logger
from sklearn.linear_model import ElasticNet
import joblib
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np


In [44]:
from pathlib import Path
import pandas as pd
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from urllib.parse import urlparse
from src.mlProject.utils.common import save_json

class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        # Load test data and trained model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # Set MLflow registry/tracking URI
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            # Make predictions
            predicted_qualities = model.predict(test_x)

            # Evaluate metrics
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log parameters and metrics
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Infer model signature
            signature = infer_signature(test_x, predicted_qualities)

            # Log the model
            if tracking_url_type_store != "file":
                # Remote tracking: register model
                model_info = mlflow.sklearn.log_model(
                    sk_model=model,
                    artifact_path="model",
                    registered_model_name="Elasticnetmodel",
                    signature=signature
                )
            else:
                # Local tracking: just log the model
                model_info = mlflow.sklearn.log_model(
                    sk_model=model,
                    artifact_path="model",
                    signature=signature
                )

            # Print the URI where model is saved
            print("Model logged at:", model_info.model_uri)


In [45]:
try:
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation_config=ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-09-07 14:23:37,107: INFO: common: yaml file:config\config.yaml loaded successfully]
[2025-09-07 14:23:37,107: INFO: common: yaml file:params.yaml loaded successfully]
[2025-09-07 14:23:37,115: INFO: common: yaml file:schema.yaml loaded successfully]
[2025-09-07 14:23:37,119: INFO: common: created directory at:artifacts]
[2025-09-07 14:23:37,120: INFO: common: created directory at:artifacts/model_evaluation]


c:\mlproject\venv\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator ElasticNet from version 1.7.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[2025-09-07 14:23:37,563: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\mlproject\venv\lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'Elasticnetmodel' already exists. Creating a new version of this model...
2025/09/07 14:23:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Elasticnetmodel,

Model logged at: runs:/562a8a9de3944c43b443e2c6d1cff612/model


2025/09/07 14:23:47 INFO mlflow.tracking._tracking_service.client: 🏃 View run sassy-gull-432 at: https://dagshub.com/saz12345/End-to-end-ML-Final-with-MLFlo.mlflow/#/experiments/0/runs/562a8a9de3944c43b443e2c6d1cff612.
2025/09/07 14:23:47 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/saz12345/End-to-end-ML-Final-with-MLFlo.mlflow/#/experiments/0.
